In [ ]:
#| default_exp api

# API

To install notebook development environment run the following command:
`conda create env -p ./.venv --file conda.env.yml`

In [ ]:
#|hide
# Development libraries, not required for package.
from nbdev.showdoc import *

Included libraries

In [ ]:
#|export
# Imports required for package
from pingme.core import Card, PingMe

In [ ]:
#|hide
# For development sets real config values which are not stored in git
import os
from pathlib import Path

if os.environ.get("PINGME_CONFIG_PATH") is None and Path("./config/config.env").is_file():
    os.environ["PINGME_CONFIG_PATH"] = "./config/config.env"

## Notebook Variables
Values are used here while not in `core` because of the values being set for this to run as a service.

In [ ]:
#|export
from pingme.core import get_config
import os
PINGME_CONFIG_PATH = os.environ.get("PINGME_CONFIG_PATH")
config = get_config(PINGME_CONFIG_PATH)

import distutils
import json
WEBHOOK_URL = config['PINGME_WEBHOOK_URL']
EMAIL_FROM = config['PINGME_EMAIL_FROM']
EMAIL_TO = config['PINGME_EMAIL_TO']
SMTP_HOST = config['PINGME_SMTP_HOST']
SMTP_PORT = config['PINGME_SMTP_PORT']
SMTP_USER = config['PINGME_SMTP_USER']
SMTP_PASSWORD = config['PINGME_SMTP_PASSWORD']
LOG_FILE = config['PINGME_LOG_FILE']
TITLE = config['PINGME_TITLE']
TEXT = config['PINGME_TEXT']
CARD_DIR = config['PINGME_CARD_DIR']
CARD_FILE = config['PINGME_CARD_FILE']
CARD_EXT = config['PINGME_CARD_EXT']
CARD_CONTEXT = json.loads(config['PINGME_CARD_CONTEXT'])
SEND_EMAIL = distutils.util.strtobool(config['PINGME_SEND_EMAIL'])
SEND_WEBHOOK = distutils.util.strtobool(config['PINGME_SEND_WEBHOOK'])
SEND_LOG_FILE = distutils.util.strtobool(config['PINGME_SEND_LOG_FILE'])

Used the [FastAPI Notebook](https://github.com/David-Lor/FastAPI_LightningTalk-Notebook/blob/master/FastAPI.ipynb) as a starting point to see how to run commands within Jupyter, then looked into running the service [inside the notebook](https://stackoverflow.com/questions/63833593/how-to-run-fastapi-uvicorn-in-google-colab) which worked but you couldn't get passed the cell. So I ended up using the [test client](https://fastapi.tiangolo.com/tutorial/testing/) which makes sense

In [ ]:
#|export
import uvicorn
from fastapi import FastAPI
from fastapi.testclient import TestClient

app = FastAPI()

@app.get("/")
async def read_main():
    return {"msg": "Hello World"}

In [ ]:
client = TestClient(app)
response = client.get("/")
print(response.status_code, response.json())

In [ ]:
#|export
from fastapi import HTTPException
@app.post("/webhook")
async def send_to_webhook(card: Card):
    try:
        notification = PingMe(card, CARD_DIR, CARD_EXT)
        response = notification.send_to_webhook(WEBHOOK_URL)
    except Exception as e:
        raise HTTPException(status_code=404, detail="Error in getting or sending card")
    return response

In [ ]:
client = TestClient(app)
response = client.post("/webhook", json={"name":"default", "context":{"title":"Title here", "text":"Text here"}})
print(response.json())

In [ ]:
client = TestClient(app)
response = client.post("/webhook", json={"name": CARD_FILE, "context": {"title": TITLE, "text": TEXT}})
# Can test when live with curl -d '{"name":"default", "context":{"title":"Title here", "text":"Text here"}}' -H 'Content-Type: application/json' -X POST http://localhost:8081/webhook

In [ ]:
#|export
from fastapi import HTTPException
@app.post("/email")
async def send_to_email(card: Card):
    try:
        notification = PingMe(card, CARD_DIR, CARD_EXT)
        response = notification.send_to_email(EMAIL_FROM, EMAIL_TO, SMTP_HOST, SMTP_PORT, SMTP_USER, SMTP_PASSWORD)
    except Exception as e:
        raise HTTPException(status_code=404, detail=f"Error in getting or sending card {e}")
    return response

In [ ]:
client = TestClient(app)
response = client.post("/email", json={"name": CARD_FILE, "context": {"title": TITLE, "text": TEXT}})
print(response.status_code, response.json())

In [ ]:
#|export
from fastapi import HTTPException
@app.post("/logfile")
async def send_to_logfile(card: Card):
    try:
        notification = PingMe(card, CARD_DIR, CARD_EXT)
        response = notification.send_to_log_file(LOG_FILE)
    except Exception as e:
        raise HTTPException(status_code=404, detail="Error in getting or sending card")
    return response

For running the web service

In [ ]:
#|export
def web_service():
    uvicorn.run(app)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()